In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from pyflann import *
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import roc_curve, auc
import cv2
import csv
import re
from sklearn.ensemble import RandomForestClassifier
import json
from tqdm import tqdm_notebook
import seaborn as sns
from sklearn.externals import joblib

# %matplotlib notebook

coordinate_header = ['FrameNumber', 'MouthHook_x', 'MouthHook_y', 'LeftMHhook_x', 'LeftMHhook_y',
                     'RightMHhook_x', 'RightMHhook_y', 'LeftDorsalOrgan_x', 'LeftDorsalOrgan_y',
                     'RightDorsalOrgan_x', 'RightDorsalOrgan_y', 'CenterBolwigOrgan_x', 'CenterBolwigOrgan_y',
                     'LeftBolwigOrgan_x', 'LeftBolwigOrgan_y', 'RightBolwigOrgan_x', 'RightBolwigOrgan_y']

distance_header = ['MouthHook', 'LeftMHhook',
                   'RightMHhook', 'LeftDorsalOrgan', 'RightDorsalOrgan',
                   'CenterBolwigOrgan', 'LeftBolwigOrgan', 'RightBolwigOrgan']

project_dir="G:/My Drive/CRG_Dropbox/AljoComputer/Temp"
dir_keypoints="../expts/KeypointsAndDescriptors/validKeyPoints"
dir_descriptors="../expts/KeypointsAndDescriptors/descriptors"

surf = cv2.xfeatures2d.SURF_create(hessianThreshold=400, nOctaves=3,nOctaveLayers=2, extended=True)

C:\Users\Shaul\Anaconda2\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
rf = joblib.load('20180809_rf_model_depth64_2015.pkl')
rf.set_params(n_jobs=1)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=64, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=350, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [3]:
meta_data_header = ['FrameNumber', 'Stage_X', 'Stage_Y', 'X', 'Y', 'Head_X', 'Head_Y', 'Tail_X', 'Tail_Y']
super_root = "G:/My Drive/CRG_Dropbox/Tracker Development (Ajinkya)/MHDO_Tracking/data/Janelia_Q2_2015/20150501_MPEG4_NoOdor/Videos/relevant/"
exp_list = ["005_20150430_174405_StageData_ContourData", "006_20150430_175243_StageData_ContourData",
            "009_20150430_182411_StageData_ContourData", "011_20150430_183916_StageData_ContourData",
            "015_20150430_191255_StageData_ContourData"]

crop_size = 512
for exp in tqdm_notebook(exp_list):
    root = super_root + exp
    for fs in os.listdir(root):
        if 'Metadata' in fs:
            meta_data_file = os.path.join(root, fs)
            meta_data = pd.read_csv(meta_data_file, sep=',', header=0, names=meta_data_header)
        if '.mp4' in fs:
            video_file = root + '/' + fs
    
    print(video_file)
    ## OpenCV object for reading video files
    cap = cv2.VideoCapture(video_file)

    ## Total number of videos in the video file
    numberFrames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(numberFrames)
    
    meta_data['pred_probab'] = np.nan
    
    if cap.isOpened():
        for row_index, row in tqdm_notebook(meta_data.iterrows()):
#             try:
            if row_index <= 30000:
                ret, originalFrame = cap.read()
                frame = originalFrame.copy()
                if frame is not None:
                    ## Crop the frame
                    crop_x = int(row['Head_X'])
                    crop_y = int(row['Head_Y'])
                    frame_crop = frame[np.max((0, crop_y-(crop_size/2))):np.min((1920, crop_y+(crop_size/2))),
                                       np.max((0, crop_x-(crop_size/2))):np.min((1920, crop_x+(crop_size/2)))]
                    kp, desc = surf.detectAndCompute(frame_crop, None)
                    
                    if desc is not None:
                        preds = rf.predict(desc)
                        meta_data.loc[row_index, 'pred_probab'] = np.mean(preds)                
#             except:
#                 print('Error')
#                 continue
            
    meta_data_file_2 = meta_data_file[:-4] + '_pred.csv'
    meta_data.to_csv(meta_data_file_2)

G:/My Drive/CRG_Dropbox/Tracker Development (Ajinkya)/MHDO_Tracking/data/Janelia_Q2_2015/20150501_MPEG4_NoOdor/Videos/relevant/005_20150430_174405_StageData_ContourData/005_20150430_174405.mp4
48280


G:/My Drive/CRG_Dropbox/Tracker Development (Ajinkya)/MHDO_Tracking/data/Janelia_Q2_2015/20150501_MPEG4_NoOdor/Videos/relevant/006_20150430_175243_StageData_ContourData/006_20150430_175243.mp4
45620


G:/My Drive/CRG_Dropbox/Tracker Development (Ajinkya)/MHDO_Tracking/data/Janelia_Q2_2015/20150501_MPEG4_NoOdor/Videos/relevant/009_20150430_182411_StageData_ContourData/009_20150430_182411.mp4
45679


G:/My Drive/CRG_Dropbox/Tracker Development (Ajinkya)/MHDO_Tracking/data/Janelia_Q2_2015/20150501_MPEG4_NoOdor/Videos/relevant/011_20150430_183916_StageData_ContourData/011_20150430_183916.mp4
52014


G:/My Drive/CRG_Dropbox/Tracker Development (Ajinkya)/MHDO_Tracking/data/Janelia_Q2_2015/20150501_MPEG4_NoOdor/Videos/relevant/015_20150430_191255_StageData_ContourData/015_20150430_191255.mp4
47183
